In [1]:
from random import random

import numpy as np
import torch
from kmeans.gensim_kmeans_plus_plus import KMeansPlusPlus

In [2]:
import torch

if torch.cuda.is_available():
    device = torch.device('cuda:0')
else:
    device = torch.device('cpu')


In [3]:
from preprocess import load_model
from sklearn import metrics

corpus, dictionary, tfidf, lsi, mSimilar = load_model("./preprocess/model_test/")
corpus_t, dictionary_t, tfidf_t, lsi_t, mSimilar_t = load_model("./preprocess/model/")

2021-04-09 20:34:08,536 : INFO : loaded corpus index from ./preprocess/model_test/corpus.mm.index
2021-04-09 20:34:08,552 : INFO : initializing cython corpus reader from ./preprocess/model_test/corpus.mm
2021-04-09 20:34:08,552 : INFO : accepted corpus with 9832 documents, 255193 features, 7002755 non-zero entries
2021-04-09 20:34:08,552 : INFO : loading Dictionary object from ./preprocess/model_test/model.dic
2021-04-09 20:34:08,723 : INFO : Dictionary lifecycle event {'fname': './preprocess/model_test/model.dic', 'datetime': '2021-04-09T20:34:08.723892', 'gensim': '4.0.1', 'python': '3.9.2 (tags/v3.9.2:1a79785, Feb 19 2021, 13:44:55) [MSC v.1928 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19041-SP0', 'event': 'loaded'}
2021-04-09 20:34:08,723 : INFO : loading TfidfModel object from ./preprocess/model_test/model.tfidf
2021-04-09 20:34:08,913 : INFO : TfidfModel lifecycle event {'fname': './preprocess/model_test/model.tfidf', 'datetime': '2021-04-09T20:34:08.913835', 'gensim': '4.0.

In [4]:
X = lsi[tfidf[corpus]]
X_t = lsi_t[tfidf_t[corpus_t]]

In [5]:
import numpy as np

X_list = []
for doc in X:
    if not doc:
        tmp_doc = [0.0 for _ in range(X.obj.num_topics)]
    else:
        tmp_doc = [i[1] for i in doc]
        if len(tmp_doc) < X.obj.num_topics:
            tmp_doc += [0 for _ in range(X.obj.num_topics - len(tmp_doc))]
    X_list.append(tmp_doc)

X_array = np.array(X_list)
X_torch = torch.from_numpy(X_array)


In [6]:
import numpy as np

X_list_t = []
for doc in X_t:
    if not doc:
        tmp_doc = [0.0 for _ in range(X_t.obj.num_topics)]
    else:
        tmp_doc = [i[1] for i in doc]
        if len(tmp_doc) < X_t.obj.num_topics:
            tmp_doc += [0 for _ in range(X_t.obj.num_topics - len(tmp_doc))]
    X_list_t.append(tmp_doc)

X_array_t = np.array(X_list_t)
X_torch_t = torch.from_numpy(X_array_t)

In [7]:
from file_utils import get_files

f_list = get_files("./corpus/test/")
l = np.array([int(i.split("\\")[-1].split("-")[0][1:]) for i in f_list])


In [14]:
from training_utils import best_map
# 0.5142392188771359
kmeans_plus_plus = KMeansPlusPlus(20, random_state=659, device=device)
label, _ = kmeans_plus_plus.fit_predict(X, mSimilar, X_array, X_torch)
acc, label_pred = best_map(l, label.numpy())
print(acc)

0.4324654190398698


In [15]:
from training_utils import best_map

kmeans_plus_plus = KMeansPlusPlus(20, random_state=202, device=device)
model = kmeans_plus_plus.fit(X_t, mSimilar_t, X_array_t, X_torch_t)
label_t = model.predict(mSimilar)
acc, _ = best_map(l, label_t.numpy())
print(acc)

0.4780309194467046


In [13]:
max_acc = 0
max_i = -1
from training_utils import best_map

# train 629
# test 98
# train test 102 202
# kpp 659 30->752
for i in range(1000):
    kmeans_plus_plus = KMeansPlusPlus(20, random_state=i, device=device)
    label, _ = kmeans_plus_plus.fit_predict(X, mSimilar, X_array, X_torch)
    acc, _ = best_map(l, label.numpy())
    if acc > max_acc:
        max_acc = acc
        max_i = i
    if acc > 0.45:
        print(i, acc)
print("max", max_i, max_acc)

1 0.4520951993490643
9 0.4503661513425549
14 0.4518917819365338
29 0.45199349064279903
37 0.45819772172497963
55 0.4869812855980472
59 0.46399511798209925
70 0.467046379170057
75 0.4636899918633035
82 0.4500610252237592
111 0.4565703824247356
113 0.4644019528071603
126 0.47375915378356387
127 0.4625711960943857
130 0.46267290480065093
132 0.4672497965825875
151 0.4653173311635476
158 0.475386493083808
181 0.45331570382424735
184 0.451586655817738
188 0.45941822620016276
210 0.4518917819365338
219 0.45382424735557364
234 0.45097640358014646
260 0.4506712774613507
268 0.4662327095199349
278 0.46379170056956875
290 0.4510781122864117
305 0.4720301057770545
309 0.48423515052888527
315 0.45829943043124494
323 0.46775834011391376
340 0.4530105777054516
342 0.4701993490642799
353 0.4527054515866558
368 0.4873881204231082
372 0.4508746948738812
373 0.47497965825874694
378 0.4822009764035802
380 0.49572823433685925
389 0.46358828315703826
397 0.4873881204231082
398 0.45829943043124494
407 0.457

In [12]:
max_acc = 0
max_i = -1
from training_utils import best_map

# 202
for i in range(2000):
    kmeans = KMeansPlusPlus(20, random_state=i, device=device)
    model = kmeans.fit(X_t, mSimilar_t, X_array_t, X_torch_t)
    label_t = model.predict(mSimilar)
    acc, _ = best_map(l, label_t.numpy())
    if acc > max_acc:
        max_acc = acc
        max_i = i
    if acc > 0.48:
        print(i, acc)
print("max", max_i, max_acc)

17 0.48260781122864116
68 0.4835231895850285
102 0.5100691619202604
133 0.48382831570382423
139 0.5066110659072417
202 0.5213588283157038
245 0.48077705451586655
269 0.48443856794141577
270 0.4846419853539463
290 0.49705044751830757
313 0.4835231895850285
333 0.49613506916192024
373 0.5049837266069975
403 0.4830146460537022
409 0.49145646867371845
411 0.48921887713588286
430 0.48260781122864116
469 0.4958299430431245
470 0.48098047192839705
472 0.4874898291293735
520 0.49125305126118796
530 0.4832180634662327
586 0.48098047192839705
603 0.48148901545972334
613 0.4921684296175753
623 0.48108218063466235
634 0.4931855166802278
635 0.48942229454841335
672 0.49288039056143207
687 0.49908462164361267
711 0.4898291293734744
725 0.4803702196908055
726 0.48565907241659884
742 0.48810008136696503
753 0.49705044751830757
754 0.49308380797396256
758 0.48962571196094384
772 0.48545565500406834
785 0.4932872253864931
786 0.48657445077298617
790 0.4818958502847844
793 0.48382831570382423
806 0.49796